# BioExplorer - Animated vasculature
![](../../bioexplorer_vasculature_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, AnimationParams
import seaborn as sns

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

In [ ]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, be.NB_MATERIALS_PER_MORPHOLOGY)

    for material_id in material_ids:
        mid = material_id % be.NB_MATERIALS_PER_MORPHOLOGY
        color = palette[mid]
        if mid == be.NEURON_MATERIAL_MITOCHONDRION:
            colors.append([1.0, 0.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_CARTOON)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.25)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            colors.append([1.0, 1.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_CARTOON)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_SOMA:
            colors.append(color)
            opacities.append(0.75)
            shading_modes.append(be.SHADING_MODE_CARTOON)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        else:
            # Membrane
            colors.append(color)
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_CARTOON)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
            
        refraction_indices.append(1.0)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

### Load vasculature

In [ ]:
def add_vasculature(seed, strength, amplitude):
    assembly_name = 'Vasculature'
    population_name = 'vasculature'
    be.remove_assembly(assembly_name)
    vasculature_assembly = be.add_assembly(assembly_name)
    vasculature_model = be.add_vasculature(
        assembly_name=assembly_name,
        population_name=population_name,
        representation=be.VASCULATURE_REPRESENTATION_SEGMENT,
        color_scheme=be.VASCULATURE_COLOR_SCHEME_REGION,
        use_sdf=True,
        sql_filter='SQRT(POW(x-345.0, 2.0) + POW(y-1357.0, 2.0) + POW(z-599.0, 2.0)) < 100',
        animation_params=AnimationParams(
            seed=1,
            position_seed=seed, position_strength=strength,
            rotation_seed=100, rotation_strength=amplitude)
    )

In [ ]:
status = core.set_renderer(
    background_color=[0,0,0],
    current='bio_explorer',subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_distance = 10.0
params.shadows = 1.0
params.soft_shadows = 0.0
params.epsilon_factor = 40.0
params.max_bounces = 10
params.show_background = True
params.exposure = 1.1
status = core.set_renderer_params(params)
status = core.set_renderer()

In [ ]:
core.set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[345.85845947265625, 1357.0596313476565, 808.9490259502797],
    target=[345.85845947265625, 1357.0596313476565, 600.9442901611328]
)

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

for i in range(0, 360):
    add_vasculature(i, 0.3, 0.1)
    model_ids = be.get_model_ids()['ids']
    offset = model_ids[0]
    set_morphology_materials(offset, 'Reds')
    status = core.set_renderer()
    mm.create_snapshot(
        renderer='bio_explorer',
        path='/tmp', base_name='%05d' % i,
        size=[960, 540], samples_per_pixel=64)
